In [1]:
def generating_percent_of_artilces(products,grouping_column,constrained_on='',constraint='',exclusions=[]):
    
    '''
    '''

    if constrained_on!='':
        products=products.query(constrained_on+' == '+constraint)
             
    df = (
        products
        .groupby(grouping_column)
        .agg({"article_id": "count"})
        .rename({"article_id": "counts"}, axis=1) 
        .query(grouping_column+" not in @exclusions")   
    )

    df=df.assign(totals=df.counts.sum())
    df=df.assign(percent_of_products=df.counts/df.totals)
    df=df.drop(['counts','totals'], axis=1)
    df=df.rename({'percent_of_products':'percent_of_products_'+constraint[1:-1]},axis=1)
    
    return df

    

In [2]:
def mergings(leftie,rightie,ons):
    '''
    '''
    if not isinstance(leftie, pd.DataFrame):
        return rightie
    else:
        return leftie.merge(rightie, how='outer',on=ons)

In [3]:
def plot_sales_breakdown(products, constrained_on,grouping_column,exclusions,ordered_list):
    '''
    '''

    new_df=''
    f = plt.figure()
    f.set_figwidth(15)

    #loop through the 
    for i in pd.unique(products[constrained_on]):
        constraint='"'+i+'"'
        
        df=generating_percent_of_artilces(products,grouping_column,constrained_on,constraint,exclusions)
        
        # reorder if we doing the reordering
        if ordered_list:
            df = df.loc[ordered_list]
        
        #function that only merges if left database exists
        new_df=mergings(new_df,df,grouping_column)
        
        new_df=new_df.fillna(0)
        
        

        plt.plot(new_df.index, new_df['percent_of_products_'+constraint[1:-1]],label=constraint[1:-1])

    plt.title('Sales by '+grouping_column)
    plt.xlabel('Color name')
    plt.ylabel('Percent of sales')   


    f.legend(loc='upper left', frameon=False)
    plt.show() 
    
    return new_df

In [6]:
def get_price_deciles(enhanced_products,category_name):
    '''
    '''
    price_deciles = (
        enhanced_products
        .query("product_group_name=='"+category_name+"'")
        .assign(deciles=lambda d: np.floor(d.price_percentile/0.1)*0.1)
        .groupby('deciles')
        .agg({'price_mean':'min'})
        .rename({'price_mean':category_name.lower().replace(' ','_')},axis=1)
    )
    return price_deciles

In [5]:
def merged_price_deciles(contrained_on,constraint,categories,columns):
    '''
    '''
    constraint='"'+constraint+'"'


    if constrained_on!='':
            ladies_enhanced_products=enhanced_products.query(constrained_on+' == '+constraint)

    # ladies_enhanced_products=enhanced_products.query("index_group_name=='Ladieswear'")

    deciles_by_category=''

    for category_name in pd.unique(products[categories]):
        category_deciles=get_price_deciles(ladies_enhanced_products,category_name)

        deciles_by_category=mergings(deciles_by_category,category_deciles,'deciles')

    if columns:
        deciles_by_category=deciles_by_category[columns]


    return deciles_by_category

In [12]:
def get_weighted_average_price_products(product_group, product_group_filter, gender_age, gender_age_filter):
    
    def w_avg(df, values, weights):
        d = df[values]
        w = df[weights]
        return (d * w).sum() / w.sum()
    
    weighted_average_price = (
            enhanced_products
            .query(f"{gender_age}=='{gender_age_filter}' & {product_group}=='{product_group_filter}'")

            .groupby('product_type_name')
            # .agg({'article_id_count':'sum'})
            .apply(w_avg, 'price_percentile', 'article_id_count')
            .reset_index(name='weighted_avg_price')
        )

    total_average_orders_per_category = (
            enhanced_products
            .query(f"{gender_age}=='{gender_age_filter}' & {product_group}=='{product_group_filter}'")

            .groupby('product_type_name')
            .agg({'article_id_count':['mean','sum']})
            # .apply(w_avg, 'price_percentile', 'article_id_count')

        )
    total_average_orders_per_category.columns=['_'.join(col) for col in total_average_orders_per_category.columns.values]

    merged_df=weighted_average_price.merge(total_average_orders_per_category,on='product_type_name',how='inner')
    
    scaled_features_sum = StandardScaler().fit_transform(please_work['article_id_count_sum'].values.reshape(-1,1))
    scaled_features_mean = StandardScaler().fit_transform(please_work['article_id_count_mean'].values.reshape(-1,1))

    scaled_features_sum_df = pd.DataFrame(scaled_features_sum, index=please_work.index, columns=['orders_sum_scaled'])
    scaled_features_mean_df = pd.DataFrame(scaled_features_mean, index=please_work.index, columns=['orders_mean_scaled'])

    adjusted_count_price=(
                merged_df
                .merge(scaled_features_sum_df,left_index=True, right_index=True)
                .merge(scaled_features_mean_df,left_index=True, right_index=True)

            )
    
    
    ax1 = adjusted_count_price.plot.scatter(x='weighted_avg_price', y='orders_mean_scaled')
            # Annotate each data point
    for i, txt in enumerate(adjusted_count_price.product_type_name):

        ax1.annotate(txt, (adjusted_count_price.weighted_avg_price.iat[i]+0.01, adjusted_count_price['orders_mean_scaled'].iat[i]))

    plt.show()
    
    ax2 = adjusted_count_price.plot.scatter(x='weighted_avg_price', y='orders_sum_scaled')
            # Annotate each data point
    for i, txt in enumerate(adjusted_count_price.product_type_name):

        ax2.annotate(txt, (adjusted_count_price.weighted_avg_price.iat[i]+0.01, adjusted_count_price['orders_sum_scaled'].iat[i]))

    plt.show()

    return adjusted_count_price